In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd
from itertools import product

In [3]:
data = pd.read_csv('/Users/shauryagulati/Developer/Kafka/cleaned_data.csv')

In [4]:
data['DateandTime'] = pd.to_datetime(data['DateandTime'])

In [5]:
data.set_index('DateandTime', inplace=True)

In [6]:
data.head()

,Date,Time,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
DateandTime,,,,,,,,,,,,,,
2004-03-10 18:00:00,2004-03-10,18:00:00,2.6,1360.00,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
2004-03-10 19:00:00,2004-03-10,19:00:00,2.0,1292.25,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2004-03-10 20:00:00,2004-03-10,20:00:00,2.2,1402.00,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
2004-03-10 21:00:00,2004-03-10,21:00:00,2.2,1375.50,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
2004-03-10 22:00:00,2004-03-10,22:00:00,1.6,1272.25,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [7]:
data.shape

(9357, 14)

In [8]:
#Extracting time-based features
data['hour'] = data.index.hour
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month

In [10]:
#Lagged features (Lag 1 and Lag 2)
data['CO_lag1'] = data['CO(GT)'].shift(1)
data['CO_lag2'] = data['CO(GT)'].shift(2)
data['NOx_lag1'] = data['NOx(GT)'].shift(1)
data['NOx_lag2'] = data['NOx(GT)'].shift(2)
data['C6H6_lag1'] = data['C6H6(GT)'].shift(1)
data['C6H6_lag2'] = data['C6H6(GT)'].shift(2)

In [11]:
#Rolling statistics (mean and std over the last 3 hours)
data['CO_rolling_mean'] = data['CO(GT)'].rolling(window=3).mean()
data['CO_rolling_std'] = data['CO(GT)'].rolling(window=3).std()
data['NOx_rolling_mean'] = data['NOx(GT)'].rolling(window=3).mean()
data['NOx_rolling_std'] = data['NOx(GT)'].rolling(window=3).std()
data['C6H6_rolling_mean'] = data['C6H6(GT)'].rolling(window=3).mean()
data['C6H6_rolling_std'] = data['C6H6(GT)'].rolling(window=3).std()

In [12]:
data.dropna(inplace=True)

In [13]:
data[['CO(GT)', 'NOx(GT)', 'C6H6(GT)', 'hour', 'day_of_week', 'month',
      'CO_lag1', 'CO_lag2', 'NOx_lag1', 'NOx_lag2', 'C6H6_lag1', 'C6H6_lag2',
      'CO_rolling_mean', 'CO_rolling_std', 'NOx_rolling_mean', 'NOx_rolling_std',
      'C6H6_rolling_mean', 'C6H6_rolling_std']].head()

,CO(GT),NOx(GT),C6H6(GT),hour,day_of_week,month,CO_lag1,CO_lag2,NOx_lag1,NOx_lag2,C6H6_lag1,C6H6_lag2,CO_rolling_mean,CO_rolling_std,NOx_rolling_mean,NOx_rolling_std,C6H6_rolling_mean,C6H6_rolling_std
DateandTime,,,,,,,,,,,,,,,,,,
2004-03-10 20:00:00,2.2,131.0,8.997817,20,2,3,2.0,2.6,103.0,166.0,9.397165,11.881723,2.266667,0.305505,133.333333,31.564748,10.092235,1.562553
2004-03-10 21:00:00,2.2,172.0,9.228796,21,2,3,2.2,2.0,131.0,103.0,8.997817,9.397165,2.133333,0.115470,135.333333,34.703506,9.207926,0.200490
2004-03-10 22:00:00,1.6,131.0,6.518224,22,2,3,2.2,2.2,172.0,131.0,9.228796,8.997817,2.000000,0.346410,144.666667,23.671361,8.248279,1.502716
2004-03-10 23:00:00,1.2,89.0,4.741012,23,2,3,1.6,2.2,131.0,172.0,6.518224,9.228796,1.666667,0.503322,130.666667,41.501004,6.829344,2.260011
2004-03-11 00:00:00,1.2,62.0,3.624399,0,3,3,1.2,1.6,89.0,131.0,4.741012,6.518224,1.333333,0.230940,94.000000,34.770677,4.961212,1.459425


In [14]:
data.to_csv("model.csv", index=False)

In [15]:
features = ['hour', 'day_of_week', 'month', 'CO_lag1', 'CO_lag2', 'NOx_lag1', 'NOx_lag2',
            'C6H6_lag1', 'C6H6_lag2', 'CO_rolling_mean', 'CO_rolling_std', 'NOx_rolling_mean',
            'NOx_rolling_std', 'C6H6_rolling_mean', 'C6H6_rolling_std', 
            'PT08.S1(CO)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)']

#Target pollutant to predict CO
target = 'CO(GT)'

In [16]:
#Train-test split (chronological)
train_size = int(0.8 * len(data))  #80% training, 20% testing
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [17]:
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [19]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("XGBoost TimeSeries Experiments")

2025/04/13 13:55:18 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost TimeSeries Experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/shauryagulati/mlruns/1', creation_time=1744566918009, experiment_id='1', last_update_time=1744566918009, lifecycle_stage='active', name='XGBoost TimeSeries Experiments', tags={}>

In [23]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'learning_rate': [0.05, 0.1],
    'reg_alpha': [0, 1],
}

best_model = {'mse': float('inf'), 'params': None, 'model_uri': None}


In [24]:
for values in product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), values))
    
    with mlflow.start_run(run_name=f"XGB {params}"):
        mlflow.set_tag("model_type", "XGBoost_Regressor")
        mlflow.set_tag("data_split", "chronological 80:20")
        
        model = xgb.XGBRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        #Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        mlflow.log_params(params)
        mlflow.log_metrics({"mse": mse, "rmse": rmse, "mae": mae, "r2": r2})
        
        #Log model
        signature = infer_signature(X_train, model.predict(X_train))
        model_info = mlflow.xgboost.log_model(
            model,
            artifact_path=f"xgb_model_{params['n_estimators']}_{params['max_depth']}",
            signature=signature,
            input_example=X_train.head(),
            registered_model_name=f"xgb_model_{params['n_estimators']}_{params['max_depth']}"
        )
        
        #Save best model
        if mse < best_model['mse']:
            best_model = {
                'mse': mse,
                'rmse': rmse,
                'mae': mae,
                'r2': r2,
                'params': params,
                'model_uri': model_info.model_uri
            }

#Print best model results
print("🏆 Best XGBoost Model (80:20 split):")
print(best_model['params'])
print(f"MSE: {best_model['mse']:.4f} | RMSE: {best_model['rmse']:.4f} | MAE: {best_model['mae']:.4f} | R2: {best_model['r2']:.4f}")

#Load best model and make predictions
loaded_model = mlflow.pyfunc.load_model(best_model['model_uri'])
predictions = loaded_model.predict(X_test)


result_df = X_test.copy()
result_df["actual_CO(GT)"] = y_test
result_df["predicted_CO(GT)"] = predictions
result_df.head()

/opt/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [13:58:38] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`,

🏃 View run XGB {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.05, 'reg_alpha': 0} at: http://127.0.0.1:5000/#/experiments/1/runs/96d7aa60cfd749ab827a4a6b52eb7344
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [13:58:44] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
Registered model 'xgb_model_50_5' already exists. Creating a new version of this model...
2025/04/13 13:58:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb_model_50_5, version 2
Created version '2' of model 'xgb_model_50_5'.


🏃 View run XGB {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.05, 'reg_alpha': 1} at: http://127.0.0.1:5000/#/experiments/1/runs/1e971dc215704ffcbb2fd88274b0ab3e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


/opt/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [13:58:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`,

...

🏃 View run XGB {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1, 'reg_alpha': 0} at: http://127.0.0.1:5000/#/experiments/1/runs/a262b9a8ef0e4c76b9715bccd09994ed
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


/opt/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [13:59:41] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`,

🏃 View run XGB {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1, 'reg_alpha': 1} at: http://127.0.0.1:5000/#/experiments/1/runs/cd83527f462747519a8b12b4a5954b68
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏆 Best XGBoost Model (80:20 split):
{'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.05, 'reg_alpha': 0}
MSE: 0.0453 | RMSE: 0.2128 | MAE: 0.1122 | R2: 0.9748


Created version '4' of model 'xgb_model_100_10'.


,hour,day_of_week,month,CO_lag1,CO_lag2,NOx_lag1,NOx_lag2,C6H6_lag1,C6H6_lag2,CO_rolling_mean,...,NOx_rolling_mean,NOx_rolling_std,C6H6_rolling_mean,C6H6_rolling_std,PT08.S1(CO),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),actual_CO(GT),predicted_CO(GT)
DateandTime,,,,,,,,,,,,,,,,,,,,,
2005-01-16 16:00:00,16,6,1,0.7,0.8,106.7,114.4,1.995293,1.892364,0.866667,...,132.266667,37.810889,2.241867,0.518777,877.25,1037.00,858.50,564.50,1.1,1.099938
2005-01-16 17:00:00,17,6,1,1.1,0.7,175.7,106.7,2.837945,1.995293,0.966667,...,154.233333,41.229278,2.696419,0.642169,892.25,1016.50,872.00,631.50,1.1,1.085428
2005-01-16 18:00:00,18,6,1,1.1,1.1,180.3,175.7,3.256020,2.837945,1.133333,...,189.333333,19.764193,3.148578,0.273243,898.75,1001.50,892.50,691.25,1.2,1.211365
2005-01-16 19:00:00,19,6,1,1.2,1.1,212.0,180.3,3.351769,3.256020,1.433333,...,241.000000,79.282974,4.564094,2.183255,1007.75,839.25,977.25,942.75,2.0,1.916879
2005-01-16 20:00:00,20,6,1,2.0,1.2,330.7,212.0,7.084494,3.351769,1.800000,...,303.800000,81.740015,5.937134,2.243703,1012.50,820.25,984.00,985.25,2.2,2.127414
